# Main notebook for battery state estimation

In [ ]:
import numpy as np
import pandas as pd
import scipy.io
import math
import os
import ntpath
import sys
import logging
import time
import sys
import random

from importlib import reload
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.layers import LSTM, Embedding, RepeatVector, TimeDistributed, Masking, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback


IS_COLAB = True

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/battery-state-estimation/battery-state-estimation/"
else:
    data_path = "../"

sys.path.append(data_path)
from data_processing.lg_dataset import LgData

### Config logging

In [ ]:
reload(logging)
logging.basicConfig(format='%(asctime)s [%(levelname)s]: %(message)s', level=logging.DEBUG, datefmt='%Y/%m/%d %H:%M:%S')

# Load Data

In [ ]:
train_names = [
    '25degC/551_LA92', 
    '25degC/551_Mixed1', 
    '25degC/551_Mixed2', 
    '25degC/551_UDDS', 
    '25degC/551_US06', 
    '25degC/552_Mixed3',

    '25degC/552_Mixed7', 
    '25degC/552_Mixed8', 
    ]
test_names = [
    '25degC/552_Mixed4', 
    '25degC/552_Mixed5', 
    '25degC/552_Mixed6', 
    ]

steps = 100

lg_data = LgData(data_path)
cycles = lg_data.get_discharge_whole_cycle(train_names, test_names, output_capacity=False)
train_x, train_y, test_x, test_y = lg_data.get_stateful_cycle(cycles, steps = steps)

train_y = lg_data.keep_only_y_end(train_y, steps, is_stateful=True)
test_y = lg_data.keep_only_y_end(test_y, steps, is_stateful=True)

# Model training

In [ ]:
# Model definition

def build_model(batch_size):
    opt = tf.keras.optimizers.Adam(lr=0.00001)

    model = Sequential()
    model.add(LSTM(512, activation='tanh',
                    return_sequences=True,
                    stateful=True,batch_size=batch_size,
                    input_shape=(train_x.shape[2], train_x.shape[3])))
    model.add(LSTM(512, activation='tanh', return_sequences=True, stateful=True))
    model.add(LSTM(256, activation='tanh', return_sequences=True, stateful=True))
    model.add(Dense(256, activation='selu'))
    model.add(Dense(128, activation='selu'))
    model.add(Dense(1, activation='linear'))
    model.summary()

    model.compile(optimizer=opt, loss='huber', metrics=['mse', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

# Other options
    #model = Sequential()
    #model.add(Masking(
    #            batch_size=batch_size,
    #            input_shape=(train_x.shape[2], train_x.shape[3])))
    #model.add(LSTM(512, activation='selu',
    #                return_sequences=True,
    #                stateful=True))
    #model.add(LSTM(512, activation='selu', return_sequences=True, stateful=True))
    #model.add(LSTM(256, activation='selu', return_sequences=False, stateful=True))
    #model.add(Dense(256, activation='selu'))
    #model.add(Dense(128, activation='selu'))
    #model.add(Dense(1, activation='linear'))
    #model.summary()

    #model = Sequential()
    #odel.add(LSTM(1024, activation='tanh',
    #                return_sequences=True,
    #                stateful=True,batch_size=batch_size,
    #                input_shape=(train_x.shape[2], train_x.shape[3])))
    #model.add(LSTM(1024, activation='tanh', return_sequences=True, stateful=True))
    #model.add(LSTM(512, activation='tanh', return_sequences=True, stateful=True))
    #model.add(Dense(256, activation='selu'))
    #model.add(Dense(128, activation='selu'))
    #model.add(Dense(64, activation='selu'))
    #model.add(Dense(1, activation='linear'))
    #model.summary()

    #model = Sequential()
    #model.add(Bidirectional(LSTM(512, activation='tanh',
    #                return_sequences=True,
    #                stateful=True), batch_size=batch_size,
    #                input_shape=(train_x.shape[2], train_x.shape[3])))
    #model.add(Bidirectional(LSTM(512, activation='tanh', return_sequences=True, stateful=True)))
    #model.add(Bidirectional(LSTM(256, activation='tanh', return_sequences=False, stateful=True)))
    #model.add(Dense(256, activation='selu'))
    #model.add(Dense(128, activation='selu'))
    #model.add(Dense(1, activation='linear'))
    #model.summary()


EXPERIMENT = "lstm_soc_lg_stateful"

experiment_name = time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + EXPERIMENT
print(experiment_name)
model = build_model(train_x.shape[0])

In [ ]:
NUM_EPOCH = 200
history = {"loss": [], "mse": [], "rmse": []}
for epoch in range(NUM_EPOCH):
    avg_history = {"loss": [], "mse": [], "rmse": []}
    shuffled_cycle_index = random.sample(range(train_x.shape[0]), train_x.shape[0])
    for chunk in range(train_x.shape[1]):
        batch_x = []
        batch_y = []
        for i in shuffled_cycle_index:
            batch_x.append(train_x[i, chunk, :, :])
            batch_y.append(train_y[i, chunk, :, :])
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        results = model.fit(batch_x, batch_y, shuffle=False, verbose=0, batch_size=train_x.shape[0])
        avg_history["loss"].append(results.history["loss"])
        avg_history["mse"].append(results.history["mse"])
        avg_history["rmse"].append(results.history["rmse"])
        print('\rChunk {}/{} - loss: {:.4e} - mse: {:.4e} - rmse: {}'.format(chunk, train_x.shape[1], np.mean(avg_history["loss"]), np.mean(avg_history["mse"]), np.mean(avg_history["rmse"])), end="")
    history["loss"].append(np.mean(avg_history["loss"]))
    history["mse"].append(np.mean(avg_history["mse"]))
    history["rmse"].append(np.mean(avg_history["rmse"]))
    print("\rEpoch {}/{} - loss: {:.4e} - mse: {:.4e} - rmse: {}".format(epoch+1, NUM_EPOCH, history["loss"][-1], history["mse"][-1], history["rmse"][-1]))
    model.reset_states()

In [ ]:
model.save(data_path + 'results/trained_model/%s.h5' % experiment_name)

hist_df = pd.DataFrame(history)
hist_csv_file = data_path + 'results/trained_model/%s_history.csv' % experiment_name
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

### Testing

In [ ]:
test_model = build_model(test_x.shape[0])
test_model.set_weights(model.get_weights())


test_history = {"loss": [], "mse": [], "rmse": []}
for chunk in range(test_x.shape[1]):
    batch_x = []
    batch_y = []
    for i in range(test_x.shape[0]):
        batch_x.append(test_x[i, chunk, :, :])
        batch_y.append(test_y[i, chunk, :, :])
    batch_x = np.array(batch_x)
    batch_y = np.array(batch_y)
    results = test_model.evaluate(batch_x, batch_y, verbose=0, batch_size=test_x.shape[0])
    test_history["loss"].append(results[0])
    test_history["mse"].append(results[1])
    test_history["rmse"].append(results[2])
    print('\rChunk {}/{} - loss: {:.4e} - mse: {:.4e} - rmse: {}'.format(chunk, test_x.shape[1], np.mean(test_history["loss"]), np.mean(test_history["mse"]), np.mean(test_history["rmse"])), end="")
print("\rEvaluation - loss: {:.4e} - mse: {:.4e} - rmse: {}".format(np.mean(test_history["loss"]), np.mean(test_history["mse"]), np.mean(test_history["rmse"])))
test_model.reset_states()

# Data Visualization

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history['loss'],
                    mode='lines', name='train'))
fig.update_layout(title='Loss trend',
                  xaxis_title='epoch',
                  yaxis_title='loss',
                  width=1400,
                  height=600)
fig.show()

In [ ]:
model.reset_states()
train_predictions = []
for chunk in range(train_x.shape[1]):
    batch_x = []
    for i in range(train_x.shape[0]):
        batch_x.append(train_x[i, chunk, :, :])
    batch_x = np.array(batch_x)
    train_predictions.append(model.predict(batch_x, batch_size=train_x.shape[0]))
train_predictions = np.array(train_predictions)

cycles = np.zeros((train_predictions.shape[1], train_predictions.shape[0], train_predictions.shape[2], 1), float)
for chunk in range(train_predictions.shape[0]):
  for cycle in range(train_predictions.shape[1]):
    cycles[cycle, chunk, :] = train_predictions[chunk, cycle, :]
train_predictions = cycles

In [ ]:
cycle_num = 0
steps_num = 10000
skip_step = 1
step_index = np.arange(cycle_num*steps_num, (cycle_num+1)*steps_num)

fig = go.Figure()
fig.add_trace(go.Scatter(x=step_index, y=train_predictions.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num:skip_step],
                    mode='lines', name='SoC predicted'))
fig.add_trace(go.Scatter(x=step_index, y=train_y.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num:skip_step],
                    mode='lines', name='SoC actual'))
fig.update_layout(title='Results on training',
                  xaxis_title='Step (x{})'.format(skip_step),
                  yaxis_title='SoC percentage',
                  width=1400,
                  height=600)
fig.show()

In [ ]:
test_model.reset_states()
test_predictions = []
for chunk in range(test_x.shape[1]):
    batch_x = []
    for i in range(test_x.shape[0]):
        batch_x.append(test_x[i, chunk, :, :])
    batch_x = np.array(batch_x)
    test_predictions.append(test_model.predict(batch_x, batch_size=test_x.shape[0]))
test_predictions = np.array(test_predictions)

cycles = np.zeros((test_predictions.shape[1], test_predictions.shape[0], test_predictions.shape[2], test_predictions.shape[3]), float)
for chunk in range(test_predictions.shape[0]):
  for cycle in range(test_predictions.shape[1]):
    cycles[cycle, chunk, :] = test_predictions[chunk, cycle, :]
test_predictions = cycles

In [ ]:
cycle_num = 0
steps_num = 800000
skip_step = 1000
step_index = np.arange(cycle_num*steps_num, (cycle_num+1)*steps_num)

fig = go.Figure()
fig.add_trace(go.Scatter(x=step_index, y=test_predictions.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num:skip_step],
                    mode='lines', name='SoC predicted'))
fig.add_trace(go.Scatter(x=step_index, y=test_y.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num:skip_step],
                    mode='lines', name='SoC actual'))
fig.update_layout(title='Results on testing',
                  xaxis_title='Step (x{})'.format(skip_step),
                  yaxis_title='SoC percentage',
                  width=1400,
                  height=600)
fig.show()